In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from IPython.display import Audio
import struct

In [2]:
%%time
from functions import *
import wave
import codecs

CPU times: user 2min 42s, sys: 141 ms, total: 2min 43s
Wall time: 2min 43s


In [3]:
%%time
p = 2 ** 521 - 1
# proof.arithmetic(False)
F = GF(p)
A = p-3
B = 1093849038073734274511112390766805569936207598951683748994586394495953116150735016013708737573759623248592132296706313309438452531591012912142327488478985984
E = EllipticCurve([F(A), F(B)])
q = E.cardinality()

CPU times: user 297 ms, sys: 0 ns, total: 297 ms
Wall time: 291 ms


In [4]:
audio = wave.open(f'audios/original/darude-sandstorm.wav')

In [5]:
data = audio.readframes(audio.getnframes())

In [6]:
data_str = str(data)[2:]

In [7]:
# %%time
P = base_point(E)
print(u'O ponto base é: ')
print(P)

O ponto base é: 
(1517170073939555601116404231070315162506193085547539896090804811897594659976473108791118523609678296607537384059906196771712086987628328416301502219380750362 : 393037236212225261621598744794531829262425464575189797754518898364708993180378355849779258145193553127146986516343352954793097948134088306354523657296209074 : 1)


In [8]:
# %%time
alice_keys = generate_keys(E, P)

print(u'Chave Privada (Alice): %s \n \n' %(alice_keys[0]))
print(u'Chave Pública (Alice):')
print(alice_keys[1])
print('\n')

Chave Privada (Alice): 3584035698607628146612153800060913868889619073063725744158040818202313120783100541672041031358065102573236077751779664339647970830865929494218301866870071103 
 

Chave Pública (Alice):
(6497628109187366942642856884699550955796013228660564962339003275841676617365675167473865329970536480780494216688273703700720947012851370862193339455205874608 : 2175746939790082179982198561823308118149023737817446661461123713796533674486360060460662908523193070421052491346183343807987549395890407122474106830121164557 : 1)




In [9]:
# %%time
bob_keys = generate_keys(E, P)

print(u'Chave Privada (Bob): %s \n \n' %(bob_keys[0]))
print(u'Chave Pública (Bob):')
print(bob_keys[1])
print('\n')

Chave Privada (Bob): 1838501654837542816306541694174738130839275077135984835012865556500122404585864111183757561849371156472040324933087487706190499983973046979551902906096249982 
 

Chave Pública (Bob):
(6699548385228509040031419257979579525805044814594356385367526167858733614251462096889781690094976394177286017295120140972600685033845184197963971791345118534 : 654433420799571198368034482878366479630085973393396456615640904166562711144759460658283955784099262958958239136079714487047201879247042606879455495670419872 : 1)




In [10]:
%%time
enc_data = encode(E, P, data_str, alice_keys[0], alice_keys[1],bob_keys[1])

CPU times: user 3h 57min 2s, sys: 1min 43s, total: 3h 58min 45s
Wall time: 4h 14min 46s


In [11]:
%%time
dec_data = decode(E, P, enc_data, alice_keys[1], bob_keys[0])

CPU times: user 1h 50min 49s, sys: 38.8 s, total: 1h 51min 28s
Wall time: 1h 56min 16s


In [12]:
%%time
dec_data_wrong1 = decode(E, P, enc_data, alice_keys[1], bob_keys[0] + 1)

CPU times: user 50min 39s, sys: 25.6 s, total: 51min 5s
Wall time: 6h 54min


In [13]:
dec_data_wrong1_b = bytes(dec_data_wrong1, 'utf-8')

In [14]:
# dec_data_wrong1_b = codecs.escape_decode(dec_data_wrong1.encode('utf-8'))[0]

In [15]:
dec_data_b = codecs.escape_decode(dec_data.encode('utf-8'))[0]

In [16]:
dec_data_b = dec_data_b[:-1]

In [17]:
dec_data == data_str

True

In [18]:
dec_data_b == data

True

In [19]:
audio.getframerate()

44100

In [20]:
dec_audio_right = wave.open("audios/decrypted/darude-sandstorm_right.wav", "w")
dec_audio_right.setnchannels(audio.getnchannels())
dec_audio_right.setsampwidth(audio.getsampwidth())
dec_audio_right.setframerate(audio.getframerate())
dec_audio_right.setcomptype('NONE', 'Not Compressed')
dec_audio_right.writeframesraw(dec_data_b)
dec_audio_right.close()

In [21]:
dec_audio_wrong = wave.open("audios/decrypted/darude-sandstorm_wrong.wav", "w")
dec_audio_wrong.setnchannels(audio.getnchannels())
dec_audio_wrong.setsampwidth(audio.getsampwidth())
dec_audio_wrong.setframerate(audio.getframerate())
dec_audio_wrong.setcomptype('NONE', 'Not Compressed')
dec_audio_wrong.writeframesraw(dec_data_wrong1_b)
dec_audio_wrong.close()